In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

import numpy as np
from random import randint
from math import floor, ceil
import codecs

In [ ]:
from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace
from bulletin.utils.normalize import normalize_text

In [ ]:
cc = CasosConfirmados()
cc.load('cc_06_01_2022', compress=True)
cc.df

In [ ]:
a = cc.df.loc[cc.df['data_comunicacao']=='2022-01-06']

In [ ]:
a = a.replace(cc.tables['metodo'].set_index('id'))
a = a.replace(cc.tables['exame'].set_index('id'))

In [ ]:
a.groupby(['metodo',a['data_diagnostico'].rename('ano').dt.year,a['data_diagnostico'].rename('mes').dt.month])['id_notifica'].count().unstack(['ano','mes']).fillna(0).astype(int).sort_index(axis=1,level=['ano','mes'])

In [ ]:
a.groupby(['exame',a['data_diagnostico'].rename('ano').dt.year,a['data_diagnostico'].rename('mes').dt.month])['id_notifica'].count().unstack(['ano','mes']).fillna(0).astype(int).sort_index(axis=1,level=['ano','mes'])

In [ ]:
a.groupby(['metodo',a['data_diagnostico'].rename('ano').dt.year,a['data_diagnostico'].rename('mes').dt.month])['id_notifica'].count().unstack(['ano','mes']).fillna(0).astype(int).sort_index(axis=1,level=['ano','mes'])

In [ ]:
# notifica.df = notifica.df.loc[((notifica.df['classificacao_final']==2)&(notifica.df['excluir_ficha']==2)&(notifica.df['status_notificacao'].isin([1,2])))]
notifica.df = notifica.df.join(notifica.tables['municipios'].set_index('ibge'), on='ibge_residencia',rsuffix='_residencia')
notifica.df = notifica.df.join(notifica.tables['municipios'].set_index('ibge'), on='ibge_unidade_notifica',rsuffix='_unidade_notifica')

In [ ]:
m = notifica.tables['municipios'].set_index('ibge')
m.loc[m['uf']=='PR'].sort_values(['rs','ibge'])

In [ ]:
surto = notifica.df.loc[(notifica.df['ibge_residencia']==410950) | (notifica.df['ibge_unidade_notifica']==410950)]
surto

In [ ]:
surto.groupby(['ibge_residencia','municipio'])[['id']].count()

In [ ]:
surto.groupby(['ibge_unidade_notifica','municipio_unidade_notifica'])[['id']].count()

In [ ]:
surto.groupby(['data_notificacao','classificacao_final'])[['id']].count().unstack().fillna(0).astype(int).droplevel(axis=1,level=0).rename(columns=notifica.tables['classificacao_final'].set_index('id').to_dict()['classificacao_final']).tail(7)

In [ ]:
surto = notifica.df.loc[(notifica.df['rs_unidade_notifica']==1) & ((notifica.df['classificacao_final']==2))]
surto.groupby(['data_notificacao','municipio_unidade_notifica'])[['id']].count().unstack().fillna(0).astype(int).droplevel(axis=1,level=0).rename(columns=notifica.tables['classificacao_final'].set_index('id').to_dict()['classificacao_final']).tail(14)